In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/NIR/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/NIR/eval.csv" /content/eval.csv

In [7]:
!cp "/content/drive/MyDrive/NIR/bald-390-20230423T052259Z-001.zip" "/content/bald-390-20230423T052259Z-001.zip"

In [8]:
!unzip "/content/bald-390-20230423T052259Z-001.zip"

Archive:  /content/bald-390-20230423T052259Z-001.zip
  inflating: bald-390/optimizer.pt   
  inflating: bald-390/training_args.bin  
  inflating: bald-390/config.json    
  inflating: bald-390/trainer_state.json  
  inflating: bald-390/rng_state.pth  
  inflating: bald-390/scheduler.pt   
  inflating: bald-390/pytorch_model.bin  


In [20]:
import pandas as pd

train = pd.read_csv("/content/train.csv", encoding='utf-8')
eval = pd.read_csv("/content/eval.csv", encoding='utf-8')

In [4]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [5]:
data = load_dataset("csv", data_files={'train': '/content/train.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-980e47bf88e2c122/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald-390")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=3,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['content'], examples['part_body'], max_length=512, truncation=True, padding="max_length")

In [11]:
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/65471 [00:00<?, ? examples/s]

In [13]:
output = trainer.predict(tokenized_data['train'].select(range(1_150, 65_471)))

In [14]:
predict_labels = np.argmax(output.predictions, axis=1)

In [21]:
# разметка 1150 первых новостей train
for row_number, index in enumerate(train.index.tolist()[1_150:]):
  train.at[index, 'relevant_part'] = int(predict_labels[row_number])

In [22]:
data = pd.concat([train, eval]).sort_values(['match_id'], ascending = True)
data

,match_id,news_id,after_match,name,team1,team2,content,part_body,len_content,relevant_part
24758,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,23 минута. Родолфо после верховой борьбы с Ант...,Антипенко получил травму в матче с «Локомотиво...,40,0
5101,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,80 минута. По многочисленным просьбам посетите...,Антипенко получил травму в матче с «Локомотиво...,28,0
12460,18925,7274151,0,Россия. Премьер-лига,Локомотив,Химки,83 минута. Дуймович приложился по ногам Никифо...,Дюрица получил травму в игре с «Химками». Защи...,53,0
35419,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,"26 минута. гол! ""ХИМКИИИИ! Йованович, замыкая ...",Антипенко получил травму в матче с «Локомотиво...,23,0
10225,18925,7274151,0,Россия. Премьер-лига,Локомотив,Химки,63 минута. СЫЧЕЕЕВ! Удар Дмитрия с семи метров...,Дюрица получил травму в игре с «Химками». Защи...,62,0
...,...,...,...,...,...,...,...,...,...,...
57865,1104865,1040572409,0,Кубок Стэнли,Питтсбург,Сан-Хосе,23 минута. Марло пытается атаковать сразу посл...,НХЛ. Форвард «Сан-Хосе» Марло забросил 65-ю ша...,175,0
60930,1104866,1040637171,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,285,1
60450,1104866,1040621229,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,285,1
20417,1104866,1040636713,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,285,1


In [23]:
data['relevant_part'].value_counts()

0    49272
1    16464
Name: relevant_part, dtype: int64

In [28]:
data.to_csv('predicted_data.csv', index=False, encoding='utf-8')

In [29]:
!zip -r "/content/predicted_data_relevant_part.zip" "/content/predicted_data.csv"
!cp "/content/predicted_data_relevant_part.zip" "/content/drive/MyDrive/NIR/predicted_data_relevant_part.zip"

  adding: content/predicted_data.csv (deflated 90%)
